In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

from keras.layers import LSTM
from keras import optimizers as opt

from mbspbs10pc import read_activations
reload(read_activations);

import mbspbs10pc.model as mbs_model
reload(mbs_model);

# Load data

In [ ]:
labelsfile = '../../tmp/1_METONLY_vs_METX/matched_CEM_table.csv'
datafile = '../../tmp/item_days_raw_data_.pkl'
dataset = load_data_labels(datafile, labelsfile)
padded_mbs_seq, padded_timestamp_seq, _ = tokenize(dataset)
maxlen = padded_mbs_seq.shape[1]

# Split in training, validation, test sets
tr_set, v_set, ts_set = train_validation_test_split(
    [padded_mbs_seq, padded_timestamp_seq], dataset['Class'],
    test_size=0.4, validation_size=0.1,
    verbose=False)

# Load model

In [7]:
model = mbs_model.build_model(mbs_input_shape=(maxlen,),
                              timestamp_input_shape=(maxlen, 1),
                              vocabulary_size=2773,
                              embedding_size=300,
                              recurrent_units=64,
                              dense_units=128,
                              bidirectional=True,
                              LSTMLayer=LSTM)

model.compile(optimizer=opt.RMSprop(lr=0.01),
              loss='binary_crossentropy',
              metrics=['acc'])
model.load_weights('../../tmp/auc_861/output_weights.h5')

# Visualize the attention weights